In [24]:
import re
from collections import deque
from math import prod
"""
|--------|
 VVVTTT
"""

"""
types:
4 = literal


"""

    


class Operator(list):
    def __repr__(self):
        ptype = getattr(self, "ptype", None)
        things = ", ".join(repr(t) for t in self)
        return f'{type(self).__name__}t{ptype}[{things}]'
    def crunch(self):
        return ptypes[self.ptype](self)
    

class L(int):
    """
    def __repr__(self):
        version = getattr(self, "version", None)
        return f'{type(self).__name__}v{self.version}({int(self)})'
    """

class Operator11(Operator): ...

class Operator15(Operator): ...

sample = "D2FE28"
class BitPuller:
    def __init__(self):
        self.bytes = deque()
        self.bits = deque()
        self.parent = self
        self.versions = 0
        self.end = False

    def empty(self):
        return not len(self.bits)

    def load(self, data):
        self.bytes.extend(bytes.fromhex(data))
        self.fill_bits()

    def get_bits(self, count):
        result = 0
        for _ in range(count):
            result = result << 1
            try:
                result = result | self.bits.popleft()
            except IndexError:
                self.end = True
                break
        return result
    
    def get_sub_bits(self, count):
        new_bp = BitPuller()
        new_bp.parent = self.parent
        current = list(self.bits)
        new_bp.bits = deque(current[:count])
        self.bits = deque(current[count:])
        return new_bp


    def fill_bits(self):
        while self.bytes:
            ondeck = self.bytes.popleft()
            for _ in range(8):
                leftbit = ondeck & 0x80
                ondeck = ondeck << 1 & 0xFF
                self.bits.append(leftbit >> 7)



def pull_literal(bp, depth):
    pad = depth * "  "
    literal = 0
    while True:
        
        keep_going = bp.get_bits(1)
        literal = literal << 4
        literal = literal | bp.get_bits(4)
        if not keep_going:
            break
    DEBUG and print (pad + f'{literal=}')
    return literal

def pull_operator(bp, depth, ptype):
    pad = depth * "  "
    length_id = bp.get_bits(1)
    if length_id:
        results = Operator11()
        results.ptype = ptype
        i = 0
        packets = bp.get_bits(11)
        while i < packets:
            i+=1
            packet = pull_packet(bp, depth)
            if packet is not None:
                results.append(packet)
            else:
                break
        DEBUG and print (pad + f'{packets} packet(s): {ptype=} {results=}')
        return results
    length = bp.get_bits(15)
    new_bp = bp.get_sub_bits(length)
    results = Operator15(pull_packets(new_bp, depth))
    results.ptype = ptype
    DEBUG and print (pad + f'{length} bits: ({len(results)}): {ptype=} {results=}')
    return results


def pull_packet(bp, depth):
    pad = depth * "  "
    depth += 1
    ver = bp.get_bits(3)
    ptype = bp.get_bits(3)
    DEBUG and print (pad + f'{ver=} {ptype=}')
    bp.parent.versions += ver
    if (not ver and not ptype) and bp.parent.end:
        return
    if ptype == 4:
        result = pull_literal(bp, depth)
    else:
        if len(bp.bits) < 12:
            return
        result = pull_operator(bp, depth, ptype)
    return result


def pull_packets(bp, depth):
    pad = depth * "  "
    results = []
    while True:
        result = pull_packet(bp, depth)
        if result is not None:
            results.append(result)
        if bp.empty():
            break
    return results


DEBUG = True
bp = BitPuller()
bp.load(sample)
pull_packets(bp, 0)

ver=6 ptype=4
  literal=2021
ver=0 ptype=0


[2021]

In [25]:
DEBUG = True
bp = BitPuller()
bp.load("9C0141080250320F1802104A08")
print(resolve(pull_packets(bp, 0)[0]))


ver=4 ptype=7
  ver=2 ptype=0
    ver=2 ptype=4
      literal=1
    ver=4 ptype=4
      literal=3
    2 packet(s): ptype=0 results=Operator11t0[1, 3]
  ver=6 ptype=1
    ver=0 ptype=4
      literal=2
    ver=2 ptype=4
      literal=2
    2 packet(s): ptype=1 results=Operator11t1[2, 2]
  80 bits: (2): ptype=7 results=Operator15t7[Operator11t0[1, 3], Operator11t1[2, 2]]
ver=0 ptype=0
results.ptype=0 results=Operator11t0[1, 3] result=4
results.ptype=1 results=Operator11t1[2, 2] result=4
results.ptype=7 results=Operator15t7[4, 4] result=1
1


In [80]:
DEBUG = True
bp = BitPuller()
bp.load("8A004A801A8002F478")
print(pull_packets(bp, 0))
print(bp.versions)

Pulling packets
Pulling packet
ver=4 ptype=2
 Pulling operator
 Pulling packet
 ver=1 ptype=2
  Pulling operator
  Pulling packet
  ver=5 ptype=2
   Pulling operator
   Pulling out 11 new bits
   Pulling packets
   Pulling packet
   ver=6 ptype=4
    Pulling a literal
    Pulling a literal literal=0
    FUCK    literal=15
   Result was Lv6(15)
  Result was Operator15v5[Lv6(15)]
  Pulling packet
  ver=0 ptype=0
 Result was Operator11v1[Operator15v5[Lv6(15)]]
 Pulling packet
 ver=0 ptype=0
Result was Operator11v4[Operator11v1[Operator15v5[Lv6(15)]]]
[Operator11v4[Operator11v1[Operator15v5[Lv6(15)]]]]
16


In [81]:
DEBUG = False
bp = BitPuller()
bp.load("620080001611562C8802118E34")
print(pull_packets(bp, 0))
print(bp.versions)

[Operator11v3[Operator15v0[Lv0(10), Lv5(11)], Operator11v1[Lv0(12), Lv3(13)]]]
12


In [82]:
DEBUG = False
bp = BitPuller()
bp.load("C0015000016115A2E0802F182340")
print(pull_packets(bp, 0))
print(bp.versions)

[Operator15v6[Operator15v0[Lv0(10), Lv6(11)], Operator11v4[Lv7(12), Lv0(13)]]]
23


In [26]:
DEBUG = False
realdeal = open('d16.input').read()
bp = BitPuller()
bp.load(realdeal)
packets = pull_packets(bp, 0)[0]
print(bp.versions)


print(resolve(packets))

873
results.ptype=0 results=Operator11t0[6, 186747] result=186753
results.ptype=3 results=Operator15t3[10, 27] result=27
results.ptype=6 results=Operator15t6[345, 114] result=0
results.ptype=1 results=Operator11t1[0, 294327] result=0
results.ptype=7 results=Operator11t7[7359896073, 94] result=0
results.ptype=1 results=Operator11t1[0, 695088758134] result=0
results.ptype=2 results=Operator15t2[46877] result=46877
results.ptype=0 results=Operator11t0[7, 13, 7] result=27
results.ptype=0 results=Operator11t0[5, 11, 13] result=29
results.ptype=5 results=Operator11t5[27, 29] result=0
results.ptype=1 results=Operator15t1[109, 0] result=0
results.ptype=6 results=Operator15t6[198032, 198032] result=0
results.ptype=1 results=Operator15t1[65117825922, 0] result=0
results.ptype=6 results=Operator15t6[66880128641, 193] result=0
results.ptype=1 results=Operator15t1[31414, 0] result=0
results.ptype=5 results=Operator11t5[93875252, 42] result=1
results.ptype=1 results=Operator11t1[30034, 1] result=300

In [22]:
gt = lambda items: 1 if items[0] > items[1] else 0
lt = lambda items: 1 if items[0] < items[1] else 0
eq = lambda items: 1 if items[0] == items[1] else 0

ptypes = {0: sum, 1: prod, 2: min, 3: max, 5: gt, 6: lt, 7: eq }

def resolve(results):
    if isinstance(results, int):
        return results
    for i, result in enumerate(results):
        if not isinstance(result, int):
            results[i] = resolve(result)
    result = results.crunch()
    print(f"{results.ptype=} {results=} {result=}")
    return results.crunch()
